# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

#Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#reads in csv
citi_weather = pd.read_csv("../output_data/cities.csv")
citi_weather = pd.DataFrame(citi_weather)
citi_weather = citi_weather.dropna()
del citi_weather['Unnamed: 0']
citi_weather
#note: this dataframe hasn't been cleaned of 'NaN' entries from weather.py. redo? or move the to_csv write?
# bad_city_weather= citi_weather.loc[(citi_weather['Lat']!="NaN") 
#                             | (citi_weather['Lng']!='NaN') 
#                             | (citi_weather['Max Temp']!='NaN')
#                             | (citi_weather['Cloudiness']!='NaN')
#                             | (citi_weather['Wind Speed']!='NaN')
#                             | (citi_weather['Country']!='NaN')
#                             | (citi_weather['Date']!='NaN')
#                            ]
# bad_city_weather.reset_index()
# bad_city_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bredasdorp,-34.5322,20.0403,292.15,88.0,98.0,1.54,ZA,1.612662e+09
1,puerto ayora,-0.7393,-90.3518,298.15,94.0,66.0,3.02,EC,1.612662e+09
2,rikitea,-23.1203,-134.9692,299.14,69.0,73.0,4.72,PF,1.612662e+09
3,mirnyy,62.5353,113.9611,245.09,88.0,100.0,1.60,RU,1.612662e+09
4,kahului,20.8947,-156.4700,299.15,50.0,40.0,5.14,US,1.612662e+09
...,...,...,...,...,...,...,...,...,...
638,fairhope,30.5230,-87.9033,283.15,100.0,90.0,3.60,US,1.612662e+09
639,schleswig,54.5167,9.5500,271.15,69.0,100.0,9.77,DE,1.612662e+09
640,teahupoo,-17.8333,-149.2667,303.15,62.0,20.0,5.66,PF,1.612662e+09
641,beringovskiy,63.0500,179.3167,251.22,90.0,61.0,5.41,RU,1.612662e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
#Store 'Lat' and 'Lng' into locations
locations = citi_weather[['Lat', 'Lng']].astype(float)
humidity = citi_weather['Humidity'].astype(float)

In [5]:
#create a heatmap layer to map
fig = gmaps.figure()

#Add layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 1)
fig.add_layer(heat_layer)

#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#null values should already be dropped
#convert degrees from Kelvin (default) to Fahrenheit
#note: this accumulates if I run multiple times and the selection process breaks. how to reset so it always works?
citi_weather['Max Temp'] = (citi_weather['Max Temp'] - 275.13)* 9/5 + 32

#criteria: 70 < max temp < 85, cloudiness < 7, wind speed < 5
vacay_city = citi_weather.loc[(citi_weather['Max Temp'] > 70) 
                             & (citi_weather['Max Temp'] < 85)
                             & (citi_weather['Cloudiness'] < 7)
                             & (citi_weather['Wind Speed'] < 5)
                             ]
#convert back to Kelvin
citi_weather['Max Temp'] = (citi_weather['Max Temp'] - 32)* 5/9 + 275.13



### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = vacay_city.copy()

hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
25,mataura,-46.1927,168.8643,78.296,44.0,5.0,0.60,NZ,1.612662e+09,
26,grand gaube,-20.0064,57.6608,71.438,86.0,2.0,0.89,MU,1.612662e+09,
96,vao,-22.6667,167.4833,74.462,80.0,0.0,2.44,NC,1.612662e+09,
136,garissa,-0.4536,39.6401,71.870,73.0,1.0,4.03,KE,1.612662e+09,
140,flinders,-34.5833,150.8552,79.430,65.0,3.0,1.79,AU,1.612662e+09,
170,faanui,-16.4833,-151.7500,79.520,65.0,0.0,4.50,PF,1.612662e+09,
187,alenquer,-1.9417,-54.7383,73.436,88.0,0.0,1.03,BR,1.612662e+09,
189,trairi,-3.2778,-39.2689,74.228,84.0,3.0,2.87,BR,1.612662e+09,
208,port blair,11.6667,92.7500,77.414,70.0,0.0,4.40,IN,1.612662e+09,
251,caico,-6.4583,-37.0978,74.642,67.0,1.0,3.77,BR,1.612662e+09,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "0, 0",
    "rankby": "distance",
    "type": "lodging",
    "locationbias": 5000,
    "keyword": "hotel",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
  
# add appropriate coordinates to params
    this_lat = row['Lat']
    this_lng = row['Lng']
    this_place = f'{this_lat},{this_lng}'
    
    params['location'] = this_place
# assemble url and make API request
    city_name = row['City']
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
#    extract results
    results = response['results']
    try:
        first_name = results[0]['name']
        print(json.dumps(first_name, indent=4))
        hotel_df.loc[index, "Hotel Name"]= first_name
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

    print("------------")

Retrieving Results for Index 25: mataura.
"Falls Hotel"
------------
Retrieving Results for Index 26: grand gaube.
"R\u00e9sidence premium Pierre & Vacances Mythic Suites & Villas"
------------
Retrieving Results for Index 96: vao.
"OURE TERA beach resort"
------------
Retrieving Results for Index 136: garissa.
"Hiddig Hotel"
------------
Retrieving Results for Index 140: flinders.
"Shellharbour Village Motel"
------------
Retrieving Results for Index 170: faanui.
"Bora Vaite Lodge"
------------
Retrieving Results for Index 187: alenquer.
"Residencial Brilhante"
------------
Retrieving Results for Index 189: trairi.
"Villa Aurora"
------------
Retrieving Results for Index 208: port blair.
"Sea Hills Hotels & Resorts"
------------
Retrieving Results for Index 251: caico.
"Regente Hotel"
------------
Retrieving Results for Index 252: deniliquin.
"Coach House Hotel-Motel"
------------
Retrieving Results for Index 259: harper.
"Grands' Guesthouse"
------------
Retrieving Results for Index 

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))